In [2]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import BarData
from ibapi.order import Order

import threading, time
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
class IBApp(EWrapper, EClient):
    def __init__(self):
        EWrapper.__init__(self)
        EClient.__init__(self, wrapper=self)

        self.data = []
        self.data_ready = threading.Event()
        self.connection_ready = threading.Event()
        self.nextOrderId = None

        self.open_orders = {}  # Stores live open orders
        self.open_orders_ready = threading.Event()

    def nextValidId(self, orderId):
        self.nextOrderId = orderId
        print(f"Connected (Next Order ID: {orderId})")
        self.connection_ready.set()

    # === Historical Data ===
    def getData(self, contract, settings, req_id=1):
        if not self.isConnected():
            raise ConnectionError("Not connected to TWS or IB Gateway.")
        self.data.clear()
        self.data_ready.clear()
        self.reqHistoricalData(req_id, contract, **settings)
        self.data_ready.wait()
        df = pd.DataFrame(self.data, columns=["Date", "Open", "High", "Low", "Close"])
        df["Date"] = pd.to_datetime(df["Date"])
        return df.set_index("Date")

    def historicalData(self, reqId, bar: BarData):
        self.data.append([bar.date, bar.open, bar.high, bar.low, bar.close])

    def historicalDataEnd(self, reqId, start, end):
        self.data_ready.set()

    # === User Utilities ===
    def createContract(self, symbol, sec_type="STK", exchange="SMART", currency="USD"):
        c = Contract()
        c.symbol, c.secType, c.exchange, c.currency = symbol, sec_type, exchange, currency
        return c

    def dataSettings(self, durationStr, barSizeSetting,
                     endDateTime='', whatToShow='TRADES', useRTH=1,
                     formatDate=1, keepUpToDate=False, chartOptions=[]):
        return {
            'endDateTime': endDateTime,
            'durationStr': durationStr,
            'barSizeSetting': barSizeSetting,
            'whatToShow': whatToShow,
            'useRTH': useRTH,
            'formatDate': formatDate,
            'keepUpToDate': keepUpToDate,
            'chartOptions': chartOptions
        }

    def displayData(self, df, rows=5):
        print(df if rows is None else df.tail(rows))

    def plotData(self, df, title="Price Chart"):
        df["Close"].plot(title=title)
        plt.tight_layout()
        plt.show()

    # === Order Management ===
    def createOrder(self, action="BUY", quantity=100, order_type="MKT", limit_price=None):
        order = Order()
        order.action = action
        order.orderType = order_type.upper()
        order.totalQuantity = quantity
        if order.orderType == "LMT" and limit_price:
            order.lmtPrice = limit_price
        # Send Order Immediately
        order.transmit = True
        # Ensure these legacy fields are NOT set
        order.eTradeOnly = False  # <- you can omit this entirely in new APIs
        order.firmQuoteOnly = False
        return order

    def placeOrderNow(self, contract, order):
        if self.nextOrderId is None:
            raise Exception("nextValidId not received yet.")
        order_id = self.nextOrderId
        self.nextOrderId += 1
        self.placeOrder(order_id, contract, order)
        print(f"New Order ID {order_id}: {order.action} {order.totalQuantity} {contract.symbol} ({order.orderType})")
        return order_id

    def listOrders(self):
        """Fetch open orders from TWS."""
        self.open_orders.clear()
        self.open_orders_ready.clear()
        self.reqOpenOrders()
        self.open_orders_ready.wait(timeout=5)
        if not self.open_orders:
            print("No live orders.")
        else:
            print("Live Orders from TWS:")
            for oid, (contract, order) in self.open_orders.items():
                print(f"ID {oid}: {order.action} {order.totalQuantity} {contract.symbol} ({order.orderType})")

    def openOrder(self, orderId, contract, order, orderState):
        self.open_orders[orderId] = (contract, order)

    def openOrderEnd(self):
        self.open_orders_ready.set()

    def modifyOrder(self, order_id, contract, new_order):
        new_order.orderId = order_id
        new_order.transmit = True
        self.placeOrder(order_id, contract, new_order)
        print(f"Modified order {order_id}")

    def cancelOrderById(self, order_id):
        self.cancelOrder(order_id)
        print(f"Cancel requested for order {order_id}")

    def cancelAllOrders(self):
        self.reqGlobalCancel()
        print("Requested cancelation of all live orders.")
    
    # === Uncomment to get Order & Execution Updates ===
    # Remove """ to enable code below
    """
    
    #=== Order Status Updates ===
    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, permId, parentId,
                    lastFillPrice, clientId, whyHeld, mktCapPrice):
        print(f"[OrderStatus] ID {orderId} | Status: {status} | "
              f"Filled: {filled}/{filled + remaining} | Avg Fill Price: {avgFillPrice}")

    # === Executed Trade Details ===
    def execDetails(self, reqId, contract, execution):
        print(f"[Execution] ID {execution.orderId} | Symbol: {contract.symbol} | "
              f"Side: {execution.side} | Price: {execution.price} | Qty: {execution.shares}")
    
    """

In [9]:
''' Demo Order Management '''
def run():
    
    app = IBApp()
    app.connect('127.0.0.1', 7497, clientId=1)

    thread = threading.Thread(target=app.run, daemon=True)
    thread.start()
    app.connection_ready.wait()

    # --- Contract ---
    symbol = "NVDA"
    contract = app.createContract(symbol)
    
    # --- Fetch Market Data ---
    # print(f"\n--- Fetch Market Data ---")
    # settings = app.dataSettings("1 M", "1 day", whatToShow="TRADES")
    # df = app.getData(contract, settings)
    # app.displayData(df)
    # app.plotData(df, title=symbol)
    
    # --- Uncomment to Use Order Management Functions ---
    # --- =========================================== ---

    # --- List Live Orders ---
    # ========================
    # print(f"\n--- List Live Orders ---")
    # app.listOrders()
    
    # --- Place MKT Order ---
    # ========================
    # print(f"\n--- Place MKT Order ---")
    # order = app.createOrder(action="BUY", quantity=1, order_type="MKT")
    # order_id = app.placeOrderNow(contract, order)

    # --- Place LMT Order ---
    # ========================
    # print(f"\n--- Place LMT Order ---")
    # order = app.createOrder(action="BUY", quantity=1, order_type="LMT", limit_price=175)
    # order_id = app.placeOrderNow(contract, order)
    
    # --- Modify MKT Order --- Use List Live Orders then update modify_order_id
    # ========================
    # print(f"\n--- Modify Order ---")
    # modify_order_id = 1
    # new_order = app.createOrder(action="BUY", quantity=5, order_type="MKT")
    # app.modifyOrder(modify_order_id, contract, new_order)
    
    # --- Modify LMT Order --- Use List Live Orders then update modify_order_id
    # ========================
    # print(f"\n--- Modify Order ---")
    # modify_order_id = 1
    # new_order = app.createOrder(action="BUY", quantity=10, order_type="LMT", limit_price=170)
    # app.modifyOrder(modify_order_id, contract, new_order)
    
    # === Cancel Order --- Use List Live Orders then update cancel_order_id
    # ========================
    # print(f"\n--- Cancel Order ---")
    # cancel_order_id = 1
    # app.cancelOrderById(cancel_order_id)
        
    # --- Cancel All Orders --
    # ========================-
    # print(f"\n--- Cancel All Orders ---")
    # app.cancelAllOrders()
    
    # --- Disconnect Cleanly ---
    print(f"\n--- Disconnect ---")
    time.sleep(1)     # Pause to ensure order messages to transmit
    app.disconnect()
    thread.join()     # wait for thread(s) to finish

if __name__ == "__main__":
    run()

ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:eufarm
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2104 Market data farm connection is OK:usopt
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


Connected (Next Order ID: 2)

--- Disconnect ---
